# Tutorial 3: The Typing Rules

*How to Assign Types to Expressions*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-typed-passages/blob/main/notebooks/03-the-typing-rules.ipynb)

---

## A Note from the Archives

> *Year 810, Capital Archives*
>
> After eight years of development, Brennis Mund finally had a complete system. The typing judgment $\Gamma \vdash M : \tau$ — read "in context Gamma, expression M has type tau" — captured precisely which passages were safe.
>
> Three rules. That was all it took. Three rules to separate the terminating from the non-terminating, the safe from the dangerous.
>
> "It's almost too simple," said his colleague Tesslyn. "Just three rules?"
>
> "Simplicity is the goal," Brennis replied. "Complex systems have hiding places for Omega."

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Explain the typing judgment $\Gamma \vdash M : \tau$
2. Apply the VAR rule to look up variables in context
3. Apply the ABS rule to type lambda abstractions
4. Apply the APP rule to type function applications
5. Construct complete typing derivations

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions_df = pd.read_csv(BASE_URL + "typed_passage_expressions.csv")
derivations_df = pd.read_csv(BASE_URL + "typing_derivations.csv")

print(f"Loaded {len(derivations_df)} derivation steps")

## 1. The Typing Judgment

The central concept is the **typing judgment**:

$$\Gamma \vdash M : \tau$$

This reads: "In context $\Gamma$, expression $M$ has type $\tau$."

The **context** $\Gamma$ is a list of variable-type pairs:
- $\Gamma = x_1:\tau_1, x_2:\tau_2, \ldots, x_n:\tau_n$
- Or $\Gamma = \emptyset$ (empty context)

The context tracks what types we've assumed for free variables.

In [ ]:
# Example typing judgments
print("Typing Judgment Examples:")
print()
print("  ∅ ⊢ λx.x : τ→τ")
print("  'In empty context, identity has type τ→τ'")
print()
print("  {x:nat} ⊢ x : nat")
print("  'If we assume x has type nat, then x has type nat'")
print()
print("  {f:nat→bool, n:nat} ⊢ (f n) : bool")
print("  'Applying f to n produces a bool'")

## 2. Rule 1: VAR (Variable Lookup)

If a variable is in the context, we can look up its type:

$$\frac{(x:\tau) \in \Gamma}{\Gamma \vdash x : \tau} \text{ (VAR)}$$

This reads: "If $x:\tau$ is in $\Gamma$, then $\Gamma \vdash x : \tau$."

In [ ]:
# VAR rule examples
var_steps = derivations_df[derivations_df['rule_applied'] == 'VAR']

print("VAR Rule Applications:")
for _, row in var_steps.head(5).iterrows():
    print(f"\n  Context: {row['context']}")
    print(f"  Variable: {row['subexpression']}")
    print(f"  Type: {row['type_assigned']}")
    print(f"  Justification: {row['justification']}")

## 3. Rule 2: ABS (Abstraction)

To type a lambda abstraction, we extend the context and type the body:

$$\frac{\Gamma, x:\tau_1 \vdash M : \tau_2}{\Gamma \vdash (\lambda x. M) : \tau_1 \to \tau_2} \text{ (ABS)}$$

This reads: "If adding $x:\tau_1$ to the context lets us derive $M : \tau_2$, then $\lambda x.M$ has type $\tau_1 \to \tau_2$."

In [ ]:
# ABS rule examples
abs_steps = derivations_df[derivations_df['rule_applied'] == 'ABS']

print("ABS Rule Applications:")
for _, row in abs_steps.head(5).iterrows():
    print(f"\n  Expression: {row['subexpression']}")
    print(f"  Type assigned: {row['type_assigned']}")
    print(f"  Justification: {row['justification']}")

## 4. Rule 3: APP (Application)

To type an application, we check that the function accepts what the argument provides:

$$\frac{\Gamma \vdash M : \tau_1 \to \tau_2 \quad \Gamma \vdash N : \tau_1}{\Gamma \vdash (M\ N) : \tau_2} \text{ (APP)}$$

This reads: "If $M$ is a function from $\tau_1$ to $\tau_2$, and $N$ has type $\tau_1$, then $(M\ N)$ has type $\tau_2$."

In [ ]:
# APP rule examples
app_steps = derivations_df[derivations_df['rule_applied'] == 'APP']

print("APP Rule Applications:")
for _, row in app_steps.head(5).iterrows():
    print(f"\n  Expression: {row['subexpression']}")
    print(f"  Result type: {row['type_assigned']}")
    print(f"  Justification: {row['justification']}")

## 5. Complete Derivation Example: Identity

Let's derive the type of $\lambda x.x$:

$$\frac{\frac{(x:\tau) \in \{x:\tau\}}{\{x:\tau\} \vdash x : \tau} \text{ (VAR)}}{\emptyset \vdash \lambda x.x : \tau \to \tau} \text{ (ABS)}$$

In [ ]:
# Show the identity derivation
identity_derivation = derivations_df[derivations_df['derivation_id'] == 'TD-001']

print("Typing derivation for λx.x:")
print()
for _, row in identity_derivation.iterrows():
    print(f"  Step {row['step_number']}: {row['rule_applied']}")
    print(f"    {row['subexpression']} : {row['type_assigned']}")
    print(f"    {row['justification']}")
    print()

## 6. Complete Derivation Example: K Combinator

Let's derive the type of $\lambda x.\lambda y.x$ (the K combinator / TRUE):

1. In context $\{x:\tau_1, y:\tau_2\}$, look up $x : \tau_1$ (VAR)
2. Abstract over $y$: $\{x:\tau_1\} \vdash \lambda y.x : \tau_2 \to \tau_1$ (ABS)
3. Abstract over $x$: $\emptyset \vdash \lambda x.\lambda y.x : \tau_1 \to \tau_2 \to \tau_1$ (ABS)

In [ ]:
# Show the K combinator derivation
k_derivation = derivations_df[derivations_df['derivation_id'] == 'TD-002']

print("Typing derivation for λx.λy.x (K combinator / TRUE):")
print()
for _, row in k_derivation.iterrows():
    print(f"  Step {row['step_number']}: {row['rule_applied']}")
    print(f"    {row['subexpression']} : {row['type_assigned']}")
    print(f"    {row['justification']}")
    print()

## 7. When Typing Fails: Self-Application

Let's try to type $(\lambda x.x\ x)$:

1. Assume $x : \tau$
2. In $(x\ x)$, we apply $x$ to $x$
3. By APP, we need $x : \tau \to \sigma$ (a function type)
4. But we also have $x : \tau$ (the argument type)
5. So $\tau = \tau \to \sigma$ — contradiction!

**No derivation exists.** The expression is untypable.

In [ ]:
# Show the failed derivation
fail_derivation = derivations_df[derivations_df['derivation_id'] == 'TD-005']

print("Attempted typing derivation for λx.x x:")
print()
for _, row in fail_derivation.iterrows():
    print(f"  Step {row['step_number']}: {row['rule_applied']}")
    print(f"    {row['subexpression']} : {row['type_assigned']}")
    print(f"    {row['justification']}")
    print()

## 8. Application Example: (λx.x) y

Let's derive the type of $(\lambda x.x)\ y$ in context $\{y:\tau\}$:

1. Derive $\lambda x.x : \tau \to \tau$ (as before)
2. Look up $y : \tau$ from context (VAR)
3. Apply: $(\lambda x.x)\ y : \tau$ (APP)

In [ ]:
# Show an application derivation
app_derivation = derivations_df[derivations_df['derivation_id'] == 'TD-003']

print("Typing derivation for (λx.x) y:")
print()
for _, row in app_derivation.iterrows():
    print(f"  Step {row['step_number']}: {row['rule_applied']}")
    print(f"    {row['subexpression']} : {row['type_assigned']}")
    print(f"    {row['justification']}")
    print()

## 9. The Rules in Summary

| Rule | Premise | Conclusion | Use |
|------|---------|------------|-----|
| VAR | $(x:\tau) \in \Gamma$ | $\Gamma \vdash x : \tau$ | Look up variables |
| ABS | $\Gamma, x:\tau_1 \vdash M : \tau_2$ | $\Gamma \vdash (\lambda x.M) : \tau_1 \to \tau_2$ | Type functions |
| APP | $\Gamma \vdash M : \tau_1 \to \tau_2$, $\Gamma \vdash N : \tau_1$ | $\Gamma \vdash (M\ N) : \tau_2$ | Type applications |

In [ ]:
# Summary of rule usage
rule_counts = derivations_df['rule_applied'].value_counts()

print("Rule usage in our derivations:")
for rule, count in rule_counts.items():
    print(f"  {rule}: {count} applications")

## Summary

In this tutorial, we learned:

1. The **typing judgment** $\Gamma \vdash M : \tau$ relates contexts, expressions, and types
2. **VAR** looks up variable types in the context
3. **ABS** types lambda abstractions by extending the context
4. **APP** types applications by matching function and argument types
5. **Typing derivations** are trees built from these three rules
6. **Self-application fails** because it requires $\tau = \tau \to \sigma$

In the next tutorial, we'll prove that well-typed programs "don't go wrong."

---

## Exercises

1. Derive the type of $\lambda f.\lambda x.(f\ x)$.

2. Derive the type of $\lambda f.\lambda g.\lambda x.(f\ (g\ x))$ (composition).

3. Show that $\lambda x.(x\ x\ x)$ is untypable. What equation would $\tau$ need to satisfy?

4. Can you type $(\lambda x.x)(\lambda y.y)$? If so, give the derivation.

---

*Next: Tutorial 4 - Type Safety*